# risklearning demo

Most, if not all, operational risk capital models assume the existence of stationary frequency and severity distributions (typically Poisson for frequencies, and a subexponential distribution such as lognormal for severities). Yet every quarter (or whenever the model is recalibrated) risk capital goes up almost without fail, either because frequencies increase, severities increase or both.

The assumption of stationary distributions is just one limitation of current approaches to operational risk modeling, but it offers a good inroad for modeling approaches beyond the usual actuarial model typical in operational capital models.

In this notebook, we give a first example of how neural networks can overcome the stationarity assumptions of traditional approaches. The hope is that this is but one of many examples showing a better way to model operational risk.

Note: What follows if very much a work in progress . . .



In [1]:
import risklearning.learning_frequency as rlf

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math

## Set up frequency distribution to generate samples

In [3]:
tenors_horizon = 365 # (Time) tenors (e.g. 1 day) per model horizon (e.g. 1 year)

h_start = 5.0 # How many model horizons of past data to train
h_end = 1.0 #How many model horizons of past data to test / validate

# Present is tenor 0, and boundary between training and testing data sets
t_start = -int(math.floor(h_start*tenors_horizon))
t_end = int(math.floor(h_end*tenors_horizon))


#% Generate Poisson-distributed events
lambda_init = 1 # intensity over tenor (e.g. day)
lambda_final = 4 # intensity over tenor (e.g. day)
n_tenors = t_end - t_start
counts = rlf.sim_counts(lambda_init, lambda_final, n_tenors)

# Build df around counts, level 1 and 2 categorization of Operational Risk events
l1s = ['Execution Delivery and Process Management']*n_tenors
l2s = ['Transaction Capture, Execution and Maintenance']*n_tenors
tenors = list(xrange(t_start, t_end))

counts_sim_df = pd.DataFrame({'t': tenors,
                              'OR Category L1': l1s, 'OR Category L2': l2s,
                              'counts': counts})


In [4]:
                    
#%% Do MLE (simple average for Poisson process
n_tenors_train = -t_start
n_tenors_test = t_end

counts_train = (counts_sim_df[counts_sim_df.t < 0]).groupby('OR Category L2').sum()
counts_test =  (counts_sim_df[counts_sim_df.t >= 0]).groupby('OR Category L2').sum()


## MLE for training data

For the Poisson distribution, the MLE of the intensity (here lambda) is just the average of the counts per model horizon. In practice, OpRisk models sometimes take a weighted average, with the weight linearly decreasing over a period of years (see e.g. "LDA at Work" by Aue and Kalkbrener).

In [5]:

lambdas_train = counts_train['counts']/n_tenors_train
lambdas_test = counts_train['counts']/n_tenors_test

bin_tops = [1,2,3,4,5,6,10,101]
# Recall that digitize (used later) defines bins by lower <= x < upper
count_tops =[count - 1 for count in bin_tops]

# Calculate bin probabilities from MLE poisson
poi_mle = stats.poisson(lambdas_train)
poi_tops = poi_mle.cdf(count_tops)
poi_tops_shifted = poi_mle.cdf(count_tops[1:])
poi_bins = np.insert(poi_tops_shifted - poi_tops[:-1], 0, poi_tops[0])

#mle_probs = pd.DataFrame({'Count Top': [t-1 for t in bin_tops], 'Probs': poi_bins})
mle_probs = pd.DataFrame(poi_bins, index = [t-1 for t in bin_tops])
mle_probs.transpose()


,0,1,2,3,4,5,9,100
0,0.107102,0.239263,0.267254,0.199012,0.111147,0.04966,0.026447,0.000114


## Prep simulated losses for neural network

For example

* Use one-hot-encoding for L1 and L2 categories (this will make more sense once we look at multiple dependent categories)
* Bin count data
* Normalize tenors (i.e. scale so that first tenor maps to -1 with 0 preserved)
* Export as numpy arrays to feed into keras / tensorflow

In [6]:
import warnings
warnings.filterwarnings('ignore') # TODO: improve slicing to avoid warnings

x_train, y_train, x_test, y_test = rlf.prep_count_data(counts_sim_df, bin_tops)


## Set up the network architecture and train

We use keras with TensorFlow backend.

Note: there has been no real attempt yet to optimize metaparameters.

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

hlayer_len = [100] # As series in anticipation of different sized layers

# Number of nodes in output layer: if series, 1, else number of cols
out_layer_len = 1 if len(y_train.shape)==1 else y_train.shape[1]
model = Sequential()
model.add(Dense(hlayer_len[0], input_shape=(x_train.shape[1],)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Default dropout parameter
model.add(Dense(hlayer_len[0]))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(hlayer_len[0]))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(out_layer_len))
model.add(Activation('softmax')) 

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# For categorical target
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train,
          batch_size=32, nb_epoch=4,
          show_accuracy=True, verbose=1,
          validation_data=(x_test, y_test))


Using TensorFlow backend.


Train on 1825 samples, validate on 364 samples
Epoch 1/4
1825/1825 [==============================] - 0s - loss: 1.8859 - val_loss: 2.0922
Epoch 2/4
1825/1825 [==============================] - 0s - loss: 1.8086 - val_loss: 2.0928
Epoch 3/4
1825/1825 [==============================] - 0s - loss: 1.7974 - val_loss: 2.0442
Epoch 4/4
1825/1825 [==============================] - 0s - loss: 1.7682 - val_loss: 2.0188


## Neural network frequency distribution

If the neural network has learned anything, we will see that the probility distribution shifts over time to higher buckets.

In [8]:
proba = model.predict_proba(x_test, batch_size=32)
proba



352/364 [============================>.] - ETA: 0s

array([[ 0.11700331,  0.1759285 ,  0.21116692, ...,  0.08428088,
         0.05115734,  0.009389  ],
       [ 0.11697587,  0.17590062,  0.21113475, ...,  0.08431186,
         0.05117796,  0.00939477],
       [ 0.1169484 ,  0.17587277,  0.21110259, ...,  0.08434284,
         0.05119858,  0.00940054],
       ..., 
       [ 0.11159051,  0.17003162,  0.20318198, ...,  0.09117258,
         0.05655767,  0.01094004],
       [ 0.11158334,  0.17002377,  0.20316948, ...,  0.09118284,
         0.05656745,  0.01094274],
       [ 0.11157613,  0.17001592,  0.20315693, ...,  0.09119311,
         0.05657726,  0.01094543]], dtype=float32)

In [9]:
nn_probs = pd.DataFrame(proba, index = range(0,t_end-1), columns = [t-1 for t in bin_tops])
# Heads (i.e. starting from present)
nn_probs.head()

,0,1,2,3,4,5,9,100
0,0.117003,0.175929,0.211167,0.212467,0.138607,0.084281,0.051157,0.009389
1,0.116976,0.175901,0.211135,0.212472,0.138633,0.084312,0.051178,0.009395
2,0.116948,0.175873,0.211103,0.212476,0.138658,0.084343,0.051199,0.009401
3,0.116921,0.175845,0.211070,0.212480,0.138684,0.084374,0.051219,0.009406
4,0.116894,0.175817,0.211038,0.212485,0.138709,0.084405,0.051240,0.009412


In [10]:
# Tails (i.e. going to end of model horizon of 1 yr)
nn_probs.tail()

,0,1,2,3,4,5,9,100
359,0.111605,0.170047,0.203207,0.210613,0.145903,0.091152,0.056538,0.010935
360,0.111598,0.170039,0.203195,0.210604,0.145917,0.091162,0.056548,0.010937
361,0.111591,0.170032,0.203182,0.210595,0.145930,0.091173,0.056558,0.010940
362,0.111583,0.170024,0.203169,0.210586,0.145944,0.091183,0.056567,0.010943
363,0.111576,0.170016,0.203157,0.210577,0.145958,0.091193,0.056577,0.010945


In [11]:
# And what MLE told us before
mle_probs.transpose()

,0,1,2,3,4,5,9,100
0,0.107102,0.239263,0.267254,0.199012,0.111147,0.04966,0.026447,0.000114


## Summary and next steps

We can see by the nn_probs data frame that the probability mass of the neural network shifts to the right, as does the underlying Poisson processes, with its intensity starting at 1 events per tenor / day at - 5 yrs and ending at 4 events per tenor / day at +1 yrs.

Next steps:

* Use better metric on generalization error that looking at probability tables (KS?)
* Optimize hyperparameters
* Simulate multiple, correlated Poisson processes
* Test non-linear non-stationarities
* Try recurrent neural network
* Try convolution network

